In [1]:
# mpu6050_temp.py
import tdwf
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Registers (integers)
PWR_MGMT1 = 0x6B
PWR_MGMT2 = 0x6C
WHO_AM_I = 0x75
SMPLRT_DIV = 0x19
CONFIG = 0x1A
TEMP_OUT_H = 0x41
TEMP_OUT_L = 0x42
SAD = 0x68

def twos_comp(val, bits=16):
    if val & (1 << (bits - 1)):
        val -= 1 << bits
    return val

def read_temperature(sht):
    # burst read 2 bytes from TEMP_OUT_H
    sht.writeread([TEMP_OUT_H], 2)  # read TEMP_OUT_H and TEMP_OUT_L
    high = sht.vals[0]
    low = sht.vals[1]
    raw = (high << 8) | low
    raw = twos_comp(raw, 16)
    # conversion: Temp (°C) = raw/340 + 36.53
    temp_c = raw / 340.0 + 36.53
    return temp_c, raw

def main():
    ad2 = tdwf.AD2()
    try:
        ad2.vdd = 3.3
        ad2.power(True)
        i2c = tdwf.I2Cbus(ad2.hdwf)
        devs = i2c.scan()
        print("I2C devices:", [hex(d) for d in devs])

        sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

        # Reset and basic configuration (wake up + enable sensors)
        sht.write([PWR_MGMT1, 0x80])  # device reset
        time.sleep(0.5)
        sht.write([PWR_MGMT1, 0x00])  # wake, clock default
        time.sleep(0.05)
        sht.write([PWR_MGMT2, 0x00])  # enable accel & gyro (temperature read unaffected by disabling)
        time.sleep(0.05)
        sht.write([CONFIG, 0x03])      # DLPF config (optional)
        time.sleep(0.01)
        sht.write([SMPLRT_DIV, 0x04])  # sample rate divider
        time.sleep(0.01)

        # Verify WHO_AM_I
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("WHO_AM_I mismatch: 0x%02x" % sht.vals[0])
            return

        # Setup plotting
        fig, ax = plt.subplots(figsize=(8,4))
        times = []
        temps = []
        start = time.time()

        running = True
        print("Press ESC in the plot window to stop.")
        def on_key(event):
            nonlocal running
            if event.key == 'escape':
                running = False
        fig.canvas.mpl_connect("key_press_event", on_key)

        while running:
            t, raw = read_temperature(sht)
            now = time.time() - start
            times.append(now)
            temps.append(t)
            print(f"{now:6.2f}s  T = {t:.2f} °C  (raw={raw})")

            ax.clear()
            ax.plot(times, temps, marker='o')
            ax.set_xlabel("Time [s]")
            ax.set_ylabel("Temperature [°C]")
            ax.grid(True)
            plt.pause(0.1)  # allows GUI event loop to run

            time.sleep(0.2)  # sample ~2-3 Hz, adjust as desired

    finally:
        print("Cleaning up hardware...")
        try:
            plt.close('all')
        except:
            pass
        ad2.power(False)
        ad2.close()
        print("AD2 Power Off. Process safely terminated.")

if __name__ == "__main__":
    main()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
I2C devices: ['0x68']
Press ESC in the plot window to stop.
  0.00s  T = 18.42 °C  (raw=-6159)
  0.52s  T = 18.53 °C  (raw=-6120)
  0.85s  T = 18.57 °C  (raw=-6108)
  1.17s  T = 18.59 °C  (raw=-6100)
  1.50s  T = 18.64 °C  (raw=-6084)
  1.83s  T = 18.66 °C  (raw=-6076)
  2.17s  T = 18.68 °C  (raw=-6069)
  2.43s  T = 18.72 °C  (raw=-6057)
  2.75s  T = 18.71 °C  (raw=-6058)
Cleaning up hardware...
Dispositivo disconnesso.
AD2 Power Off. Process safely terminated.


KeyboardInterrupt: 